In [1]:
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Dynamically find the full path to the DB
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
db_path = os.path.join(project_root, "data", "games.db")

# Connect to the DB
conn = sqlite3.connect(db_path)

# Run the query
query = """
SELECT
    p.date,
    p.home_team,
    p.away_team,
    p.predicted_home_score,
    p.predicted_away_score,
    p.predicted_diff,
    p.win_probability,
    p.conf_low,
    p.conf_high,
    g.home_score AS actual_home_score,
    g.away_score AS actual_away_score,
    (g.home_score - g.away_score) AS actual_diff
FROM predictions p
JOIN games g
  ON p.date = g.date
 AND p.home_team = g.home_team
 AND p.away_team = g.away_team
WHERE g.home_score IS NOT NULL AND g.away_score IS NOT NULL
"""

df = pd.read_sql(query, conn)
conn.close()

# Compute metrics
df["error"] = (df["predicted_diff"] - df["actual_diff"]).abs()
df["winner_correct"] = (
    ((df["predicted_diff"] > 0) & (df["actual_diff"] > 0)) |
    ((df["predicted_diff"] < 0) & (df["actual_diff"] < 0))
)

# Show recent games
df.sort_values("date", ascending=False).head(10)


,date,home_team,away_team,predicted_home_score,predicted_away_score,predicted_diff,win_probability,conf_low,conf_high,actual_home_score,actual_away_score,actual_diff,error,winner_correct
13,2025-07-10,Los Angeles Sparks,Minnesota Lynx,87,60,27.598609,1.0000,25.973491,29.259105,82,91,-9,36.598609,False
14,2025-07-10,Washington Mystics,Las Vegas Aces,80,78,1.400144,0.9237,-0.231276,3.086998,70,68,2,0.599856,True
9,2025-07-09,Connecticut Sun,Seattle Storm,85,75,9.599655,1.0000,7.964294,11.250980,93,83,10,0.400345,True
10,2025-07-09,Indiana Fever,Golden State Valkyries,80,76,4.000039,0.9998,2.351502,5.614353,61,80,-19,23.000039,False
11,2025-07-09,Phoenix Mercury,Minnesota Lynx,96,73,22.998852,1.0000,21.345511,24.619376,79,71,8,14.998852,True
12,2025-07-09,Chicago Sky,Dallas Wings,89,77,11.399554,1.0000,9.742367,13.045967,87,76,11,0.399554,True
7,2025-07-08,Washington Mystics,Chicago Sky,83,82,0.400197,0.6577,-1.256049,2.072495,81,79,2,1.599803,True
8,2025-07-08,New York Liberty,Las Vegas Aces,87,75,11.199548,1.0000,9.576149,12.817550,87,78,9,2.199548,True
5,2025-07-07,Atlanta Dream,Golden State Valkyries,86,72,14.399383,1.0000,12.722385,16.083356,90,81,9,5.399383,True
6,2025-07-07,Phoenix Mercury,Dallas Wings,92,81,10.999528,1.0000,9.399017,12.665041,102,72,30,19.000472,True
